In [31]:
import optuna
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

Data Preparation

In [32]:
db = pd.read_excel("Retail-Supply-Chain-Sales-Dataset.xlsx")
target['Duration'] =db['Ship Date']-db['Order Date']
target['Ship Day'] = db['Ship Date'].dt.dayofweek
target['Ship Month']= db['Ship Date'].dt.month
target['Order Month'] = db['Order Date'].dt.month
target['Returned']=db['Returned'].map({'Yes':1,'Not':0})

def extract_brand(product_name):
    if isinstance(product_name,str):
        return product_name.split()[0].strip(",").title()
    return "Unknown"
target['Brand']=target['Product Name'].apply(extract_brand)
target['Duration']=target['Duration'].dt.days
encoding_cols = ['Ship Mode','Segment','City','Sub-Category','Brand','Region']

for cols in encoding_cols:
    for cols in encoding_cols:
        le = LabelEncoder()
        target[cols] = le.fit_transform(target[cols])
target.drop(columns=['Product Name','Customer Name','Sales'],inplace= True)

X = target.drop(columns=['Returned'])
y = target['Returned']
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
x_est,x_final,y_est,y_final = train_test_split(x_train,y_train,test_size=0.2,random_state=42,stratify =y_train )


KeyError: 'Product Name'

Optuna Tuning For XGBoost

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 250, 350]),
        'max_depth': trial.suggest_categorical('max_depth', [5, 15, 25]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.1, 0.3]),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
        'eval_metric': 'logloss',
        'random_state': 42
    }
    model = XGBClassifier(**params)
    model.fit(x_train, y_train)
    preds = model.predict(x_test)
    return accuracy_score(y_test, preds)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print("\n🔧 Best XGBoost Parameters:")
print(study.best_params_)

GridSearchCV : LightGBM

In [24]:
lgbm_param={
    'n_estimators': [100, 300],
    'learning_rate': [0.01, 0.1],
    'max_depth': [4, 6, 8],
    'num_leaves': [15, 31, 63],
    'min_child_samples': [10, 20],
    'subsample': [0.8, 1.0]
}

LGBM_model=LGBMClassifier(random_seed=42)
lgbm_grid=GridSearchCV(
    estimator=LGBM_model,
    scoring='accuracy',
    cv=3,
    n_jobs=-1,
    param_grid=lgbm_param
)
lgbm_grid.fit(x_train,y_train)
print('Best Params: ',lgbm_grid.best_params_)

ValueError: 
All the 432 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
432 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hp\anaconda3\Lib\site-packages\lightgbm\sklearn.py", line 1560, in fit
    super().fit(
  File "C:\Users\hp\anaconda3\Lib\site-packages\lightgbm\sklearn.py", line 1049, in fit
    self._Booster = train(
                    ^^^^^^
  File "C:\Users\hp\anaconda3\Lib\site-packages\lightgbm\engine.py", line 297, in train
    booster = Booster(params=params, train_set=train_set)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp\anacond

GridSearchCV : Random Forest

In [ ]:
rf_params={
    'n_estimators':[100,300],
    'max_depth':[10,20,None],
    'min_samples_split':[2,5],
    'min_samples_leaf':[1,2],
    'bootstrap':[True,False]
}
rf_search=RandomForestClassifier(random_state=42)
rf_best=GridSearchCV(
    estimator= rf_search,
    param_grid= rf_params,
    cv=3,
    scoring='accuracy',
    n_jobs=-1
)
rf_best.fit(x_train,y_train)
print("Random Forest: ",rf_best.best_params_)

GridSearchCV : CATBoost

In [ ]:
cat_param_grid = {
    'iterations':[100,300],
    'depth':[4,6,8],
    'learning_rate':[0.01,0.1],
    'border_count':[32,64],
    'verbose':[0]
}
CAT_model = CatBoostClassifier(random_seed=42)
CAT_grid=GridSearchCV(
    estimator=CAT_model,
    param_grid=cat_param_grid,
    scoring='accuracy',
    cv=3,
    n_jobs=-1,
    verbose=1
)
CAT_grid.fit(x_train,y_train)
print("Best Case: ",CAT_grid.best_params_)